In [1]:
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 
import os 
import cv2
import pickle
import pandas as pd
import itertools
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

%matplotlib inline

2025-02-21 11:19:24.624288: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 11:19:24.703164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740129564.765491 4027577 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740129564.783845 4027577 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 11:19:24.843035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
cm_plot_labels = ['Heart','Rectangle','Oval','Round', 'Square', 'Triangle']
fig_path = './images/'

def create_confusion_matrix(y_test_labels, y_preds, classes, title='Confusion Matrix', 
                            normalize=False, cmap=plt.cm.Blues):

    cm = confusion_matrix(y_test_labels, y_preds)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    plt.figure(figsize=(16,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize = 15)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f"{fig_path+title}.png");    # for saving images to .png file
    plt.show()


In [6]:
def compare_misclass(df, predict, fig_title='Comparing Misclassification', ax_title=''):
    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(15,5))
    fig.suptitle(fig_title, fontsize=16)
    ax[0].hist(df.y_actual, bins=9, color='lightcoral')
    ax[0].set_xticks(range(0,6))
    ax[0].set_xticklabels(cm_plot_labels)
    ax[0].set_title(f'{ax_title}\n ACTUAL CLASS')
    ax[1].hist(df[predict], bins=9, color='mediumturquoise')
    ax[1].set_xticks(range(0,6))
    ax[1].set_xticklabels(cm_plot_labels)
    ax[1].set_title(f'{ax_title}\n PREDICTED CLASS');
    plt.savefig(f"{fig_path+fig_title}.png");

In [7]:
def plot_misclass_img_proba(df, list_index):
  for i in list_index:
    img = X_test[i] * 255
    img = np.asarray(img, int)
    label = y_label_dict[y_actual[i]]

    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(img, cmap='gray')
    plt.title(label, size=16)
    plt.subplot(1,3,2)
    df.loc[i, ['heart_s', 'rectangle_s', 'oval_s', 'round_s','square_s','triangle_s']].plot(kind='bar', color='pink')
    plt.title('Probabilities - CNN from scratch', fontsize=14, y=1.01)
    plt.subplot(1,3,3)
    df.loc[i, ['heart_t', 'rectangle_t', 'oval_t', 'round_t','square_t','triangle_t']].plot(kind='bar', color='plum')
    plt.title('Probabilities - CNN transfer learning', fontsize=14, y=1.01);

In [8]:
path = './pickle_out/rgb/'

X_train = np.asarray(pickle.load(open(path + "pickle_out_rgbX_train_rgb.pickle","rb")))
y_train = np.asarray(pickle.load(open(path + "pickle_out_rgby_train_rgb.pickle","rb")))
X_test = np.asarray(pickle.load(open(path + "pickle_out_rgbX_test_rgb.pickle","rb")))
y_test = np.asarray(pickle.load(open(path + "pickle_out_rgby_test_rgb.pickle","rb")))

In [9]:
print("Data Summary")
print("--------------------")
print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print("--------------------")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")

Data Summary
--------------------
X_train shape (3620, 224, 224, 3)
y_train shape (3620, 6)
--------------------
X_test shape (1204, 224, 224, 3)
y_test shape (1204, 6)


In [ ]:
# New model from scratch

scratch_path = './SavedModels/AGMT.keras'
scratch_file = scratch_path + 'agmt_feb2025_v2.keras'
mod_scratch = tf.keras.models.load_model(scratch_file)

In [ ]:
# Transfer Learning model from VGG-Face

transfer_path = './SavedModels/VGGFACE.keras'
transfer_file = transfer_path + 'vgg16-face-2.keras'
mod_transfer = tf.keras.models.load_model(transfer_file)

In [ ]:
# Make predictions
y_actual = np.argmax(y_test, axis=-1)
y_predict_scratch = np.argmax(mod_scratch.predict(X_test), axis=1)
y_predict_transfer = np.argmax(mod_transfer.predict(X_test), axis=1)

In [ ]:
# Generate predict probabilities
predict_proba_scratch = mod_scratch.predict(X_test)
predict_proba_transfer = mod_transfer.predict(X_test)

In [ ]:
mod_scratch.evaluate(X_test, y_test)

In [ ]:
mod_transfer.evaluate(X_test, y_test)

In [ ]:
create_confusion_matrix(y_actual, y_predict_scratch, cm_plot_labels, 
                        title='Confusion Matrix - CNN Model without Transfer Learning', 
                        normalize=True, cmap=plt.cm.Blues)

In [ ]:
create_confusion_matrix(y_actual, y_predict_transfer, cm_plot_labels, 
                        title='Confusion Matrix - CNN Model with Transfer Learning', 
                        normalize=True, cmap=plt.cm.Blues)

In [ ]:
def create_df(y_actual, y_predict, predict_proba, X_test, y_test):
  '''this function creates a dataframe with predictions and probabilities of a model'''
  # generate predictions

  actual = pd.DataFrame(y_actual, columns=['y_actual'])
  predict = pd.DataFrame(y_predict, columns=['y_predict'])

  # generate prediction probabilities 

  probability_list = []
  for i, item in enumerate(predict_proba):
    probabilities = {}
    probabilities['heart'] = round(item[0] * 100,2)
    probabilities['rectangle'] = round(item[1] * 100,2)
    probabilities['oval'] = round(item[2] *100,2)
    probabilities['round'] = round(item[3] *100,2)
    probabilities['square'] = round(item[4] *100,2)
    probabilities['triangle'] = round(item[5] *100,2)
    probability_list.append(probabilities)
  proba = pd.DataFrame(probability_list)

  # create dataframe
  df = pd.concat([actual, predict, proba],axis=1)
  
  return df


In [ ]:
scratch = create_df(y_actual, y_predict_scratch, predict_proba_scratch, X_test, y_test)

transfer = create_df(y_actual, y_predict_transfer, predict_proba_transfer, X_test, y_test)

In [ ]:
# dataframe for model built from scratch
scratch.columns = [x + '_s' for x in scratch.columns]
scratch.head()

In [ ]:
# dataframe for model with transfer learning
transfer.columns = [x + '_t' for x in transfer.columns]
transfer.head()

In [ ]:
# combine the data frame for evaluations
evaluations = pd.concat([scratch, transfer], axis=1)

# drop one of the y_actual as they are the same
evaluations = evaluations.drop(columns='y_actual_t')
evaluations = evaluations.rename(columns={'y_actual_s': 'y_actual'})

# create columns with actual labels
evaluations['actual'] = evaluations['y_actual'].map(y_label_dict)
evaluations['s_predict'] = evaluations['y_predict_s'].map(y_label_dict)
evaluations['t_predict'] = evaluations['y_predict_t'].map(y_label_dict)

# create new columns to detect where the 2 models misclassify, and the differences in predictions
evaluations['predict_diff'] = evaluations['y_predict_s'] - evaluations['y_predict_t']
evaluations['t_misclass'] = evaluations['y_actual'] - evaluations['y_predict_t']
evaluations['s_misclass'] = evaluations['y_actual'] - evaluations['y_predict_s']

evaluations

In [ ]:
print('SUMMARY OF MISCLASSIFICATION')

correct = evaluations[(evaluations.y_predict_t == evaluations.y_actual) & (evaluations.y_predict_s == evaluations.y_actual)]
print(f'\nNumber of images that both models predicted correctly are {correct.shape[0]}')

misclass = evaluations[(evaluations.y_predict_t != evaluations.y_actual) | (evaluations.y_predict_s != evaluations.y_actual)]
print(f'Number of images that are misclassified by either of the models are {misclass.shape[0]}')
print(f'\n--------------------------------------------------------------------------------------\n')

# where both models predicted incorrectly
both_wrong = misclass[(misclass.s_misclass != 0) & (misclass.t_misclass != 0)]
both_wrong_same = misclass[((misclass.s_misclass != 0) & (misclass.t_misclass != 0)) & (misclass.t_misclass == misclass.s_misclass)]
both_wrong_different = misclass[((misclass.s_misclass != 0) & (misclass.t_misclass != 0)) & (misclass.t_misclass != misclass.s_misclass)]
print(f"Both models predicted incorrectly: {both_wrong.shape[0]}")
print(f"Same class: {both_wrong_same.shape[0]}")
print(f"Different class: {both_wrong_different.shape[0]}")
print(f'\n--------------------------------------------------------------------------------------\n')

# where transfer learning improved predictions vs. the model from scratch
t_misclass = misclass[(misclass.t_misclass != 0)]
print(f"Transfer learning predicted incorrectly: {misclass[(misclass.t_misclass != 0)].shape[0]}")
print(f"Model from scratch predicted correctly: {misclass[(misclass.t_misclass != 0) & (misclass.s_misclass == 0)].shape[0]}")
print(f'\n--------------------------------------------------------------------------------------\n')

# where model from scratch predicted correctly, but the transfer learning did not
s_misclass = misclass[(misclass.s_misclass != 0)]
print(f"Transfer learning predicted incorrectly: {misclass[(misclass.s_misclass != 0)].shape[0]}")
print(f"Model from scratch predicted correctly: {misclass[(misclass.s_misclass != 0) & (misclass.t_misclass == 0)].shape[0]}")
print(f'\n--------------------------------------------------------------------------------------\n')


In [ ]:
compare_misclass(t_misclass, 'y_predict_t', fig_title='Missclassified:  Transfer Learning Model only (32 images)', ax_title='')


In [ ]:
compare_misclass(s_misclass, 'y_predict_s', fig_title='Missclassified:  NON-Transfer Learning Model only (190 images)', ax_title='')


In [ ]:
# identify and plot 20 images where both models misclassify, but incorrectly predicted different classes.

both_wrong_same_index = both_wrong_same[(both_wrong_same.actual == 'Oval')].head(30).index
plot_misclass_img_proba(both_wrong_same[(both_wrong_same.actual == 'Oval')].head(30), both_wrong_same_index)

In [ ]:
# identify and plot 20 images where both models misclassify, but incorrectly predicted different classes.

both_wrong_different_index = both_wrong_different.head(20).index
plot_misclass_img_proba(both_wrong_different.head(20), both_wrong_different_index)

In [ ]:


# identify and plot 20 images where Transfer Learning misclassifies, but Model from scratch correctly predicted.

t_misclass_index = t_misclass[(t_misclass.s_misclass == 0)].head(20).index
plot_misclass_img_proba(t_misclass[(t_misclass.s_misclass == 0)].head(20), t_misclass_index)



In [ ]:
# filter where CNN from scratch misclassification, while Transfer Learning has 100% probability on the predicted class
mask_100 = ((s_misclass.heart_t == 100) | (s_misclass.rectangle_t == 100) | (s_misclass.oval_t == 100) |
            (s_misclass.round_t == 100) | (s_misclass.square_t == 100) | (s_misclass.triangle_t == 100))
s_misclass_t100 = s_misclass[(mask_100)]
s_misclass_t100.groupby('actual')['y_actual'].count().plot(kind='bar', color='skyblue')
plt.title('TRANSFER LEARNING\nImprove misclassification to 100% predict_probability\n(No. of Images)');
plt.savefig(f"transfer_100.png");


In [ ]:
s_misclass_t100_index = s_misclass_t100.head(20).index
plot_misclass_img_proba(s_misclass_t100.head(20), s_misclass_t100_index)